# FedML usage example in IBM Watsonx.ai 

In [ ]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

## Pre-req steps required in IBM Watsonx.ai to load the needed config files from workspace Data Assets

In order to import config files from the workspace, you will need to use the code snippet tool to add the credentials of your S3 bucket to the notebook. The credentials remain the same between the assets loaded in, 
but the `'FILE'` parameter changes to the name of the asset you are trying to use.

These cells that contain the credentials should be marked with the following:

```
# @hidden_cell
# The following code contains metadata for a file in your project storage.
# You might want to remove secret properties before you share your notebook.
```

This will provide the option to keep the sensitive cells hidden upon sharing the link to the notebook instance. Note, by default the inserted code labels the dict as `storage_metadata`.

For more information on how to get the credentials, please refer to these [IBM Waston Studio docs](https://www.ibm.com/docs/en/cloud-paks/cp-data/4.8.x?topic=scripts-loading-accessing-data-in-notebook). When inserting the code to cell in the notebook instance, you will want to select `load as credentials`. Ensure you load this code snippet to a fresh cell.

In [ ]:
# @hidden_cell
# The following code contains metadata for a file in your project storage.
# You might want to remove secret properties before you share your notebook.

dsp_config_json = {
    'IAM_SERVICE_ID': <service_id>,
    'IBM_API_KEY_ID': <key_id>,
    'ENDPOINT': <s3_endpoint>,
    'IBM_AUTH_ENDPOINT': <auth_endpoint>,
    'BUCKET': <bucket_name>,
    'FILE': <file_name>
}

The cell below only needs to be run with the first credentials, as the `'FILE'` parameter (which is the only one changing on each asset) is not used.

In [ ]:
client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=dsp_config_json['IBM_API_KEY_ID'],
    ibm_service_instance_id=dsp_config_json['IAM_SERVICE_ID'],
    ibm_auth_endpoint=dsp_config_json['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=dsp_config_json['ENDPOINT'])

In [ ]:
client.download_file(Bucket=dsp_config_json['BUCKET'], Key=dsp_config_json['FILE'], Filename=dsp_config_json['FILE'])

Note the variable name change below from `dsp_config_json` to `aic_config`. We advise to do the same for each new JSON asset that is imported into the notebook.

In [ ]:
# @hidden_cell
# The following code contains metadata for a file in your project storage.
# You might want to remove secret properties before you share your notebook.

aic_config = {
    'IAM_SERVICE_ID': <service_id>,
    'IBM_API_KEY_ID': <key_id>,
    'ENDPOINT': <s3_endpoint>,
    'IBM_AUTH_ENDPOINT': <auth_endpoint>,
    'BUCKET': <bucket_name>,
    'FILE': <file_name>
}

In [ ]:
client.download_file(Bucket=aic_config['BUCKET'], Key=aic_config['FILE'], Filename=aic_config['FILE'])

In [ ]:
# @hidden_cell
# The following code contains metadata for a file in your project storage.
# You might want to remove secret properties before you share your notebook.

docker_registry = {
    'IAM_SERVICE_ID': <service_id>,
    'IBM_API_KEY_ID': <key_id>,
    'ENDPOINT': <s3_endpoint>,
    'IBM_AUTH_ENDPOINT': <auth_endpoint>,
    'BUCKET': <bucket_name>,
    'FILE': <file_name>
}

In [ ]:
client.download_file(Bucket=docker_registry['BUCKET'], Key=docker_registry['FILE'], Filename=docker_registry['FILE'])

In [ ]:
# @hidden_cell
# The following code contains metadata for a file in your project storage.
# You might want to remove secret properties before you share your notebook.

github_info = {
    'IAM_SERVICE_ID': <service_id>,
    'IBM_API_KEY_ID': <key_id>,
    'ENDPOINT': <s3_endpoint>,
    'IBM_AUTH_ENDPOINT': <auth_endpoint>,
    'BUCKET': <bucket_name>,
    'FILE': <file_name>
}

In [ ]:
client.download_file(Bucket=github_info['BUCKET'], Key=github_info['FILE'], Filename=github_info['FILE'])

# Pip install  FedML 

In [ ]:
pip install fedml-dsp

# Import FedML

In [ ]:
import fedml_dsp
from fedml_dsp import DbConnection

In [ ]:
import pandas as pd
import seaborn as sns

# Connect to SAP Datasphere 

The string below should be whatever your dsp configuration JSON file is called. You can run an `ls` after loading in the files from above to verify what you have labeled it.

In [ ]:
db = DbConnection(url="./config.json")

# Data discovery & exploration (of Quality SAP semantic data )  with single line of code.

In [ ]:
db.get_schema_views()

# Read data from SAP Datasphere semantic models 

In [ ]:
data = db.get_data_with_headers("IRIS_VIEW")
iris_df = pd.DataFrame(data[0], columns=data[1])
iris_df.head()

# Example usage of Reading data directly into PySpark DataFrame. 

In [ ]:
query = "select * from \"SCE\".\"IRIS_VIEW\" limit 5"
db.execute_query_pyspark(query)

# Continue with regular ML flow tasks; eg; Scikit-Learn linear regression model training

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
iris_clean_df = iris_df.dropna()
iris_clean_df.drop("species", axis=1, inplace=True)
iris_clean_df.head()

In [ ]:
label_col = "petal_length"
y = iris_clean_df[label_col]
iris_clean_df.drop(label_col, axis=1, inplace=True)
print(np.shape(iris_clean_df), np.shape(y))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(iris_clean_df, y, test_size=0.3)

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
predictions = lr.predict(X_test)
predictions

In [ ]:
print('Mean Absolute Error:', mean_absolute_error(y_test, predictions))
print('Mean Squared Error:', mean_squared_error(y_test, predictions))
print('Mean Root Squared Error:', np.sqrt(mean_squared_error(y_test, predictions)))

# Example: SAP AI Core Deployment 

For more information on AI Core functions, please refer to our [AI Core function docs](https://github.wdf.sap.corp/SCE/FedML-Generic/blob/main/fedml-dsp.md).

In [ ]:
APPLICATION_NAME = "watson-x-testing1"

In [ ]:
from fedml_dsp import Fedml 

Loading in AI Core credentials:

In [ ]:
try:
    fedml = Fedml(aic_service_key='aic_service_key.json')
except:
    raise Exception("Fedml constructor failed.")

Onboarding AI Core resources:

In [ ]:
try:
    fedml.onboard_ai_core(
                        create_resource_group=False, # set this to True if the GitHub repo is not already onboarded
                        resource_group='default',
                        onboard_new_repo=True,
                        github_info_path="./github_info.json",
                        secret_path="./docker_registry.json"
                    )
except:
    raise Exception("AI Core resource onboarding failed.")

Registering the application:

In [ ]:
try:
    application_details = {
        "application_name": APPLICATION_NAME, 
        "revision": "HEAD", 
        "repository_url": <repository_url>,
        "path": <path_to_yaml_config>
    }

    fedml.register_application(application_details=application_details)
except:
    raise Exception("Application registration failed.")

Deployment to AI Core:

In [ ]:
try:
    deployment_config = {
        "name": APPLICATION_NAME, 
        "resource_group": "default", 
        "scenario_id": <scenario_id>,
        "executable_id": <executable_id>
    }

    endpoint = fedml.ai_core_deploy(deployment_config=deployment_config)
except:
    raise Exception("Deployment to AI Core failed")

Data formatting and inferencing the endpoint:

In [ ]:
def get_test_data():
    """
    Grab test data for inferencing the model.
    """
    org_data = db.get_data_with_headers(table_name="BREASTCANCER_VIEW", size=1)
    org_data = pd.DataFrame(org_data[0], columns=org_data[1])
    org_data = org_data.sample(frac=1).reset_index(drop=True)
    org_data = org_data[500:]
    org_data.fillna(0, inplace=True)

    y = org_data['diagnosis']
    X = org_data.drop(['diagnosis'], axis=1)

    return X

In [ ]:
try:
    X = get_test_data()

    headers = {
        "Authorization": fedml.get_ai_core_token(),
        "ai-resource-group": "default",
        "Content-Type": "text/csv"}

    temp = X.to_json()
    print()
    print(X.to_json())
    print()

    response_data = fedml.ai_core_inference(
        endpoint=endpoint,
        headers=headers,
        body=X.to_json()
    )

    print()
    print("Response data:")
    print(response_data)
except:
    raise Exception("Model inferencing failed.")